In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv("../data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
df1 = pd.read_csv("../data/HistoricalData_BTC.csv", parse_dates=["Date"])

# process dataframe from kaggle
df = df.dropna()

# timestamp to datetime
df["DateTime"] = pd.to_datetime(df['Timestamp'],unit='s')
df["Date"] = df["DateTime"].dt.date
df["Time"] = df["DateTime"].dt.time

# keep only useful columns and add a column indicate weather the close value is acctual or predict
df = df[["Date","Time", "Close"]]
df["Real"] = True

# get the date closing price then sort and reset index
df = df.groupby("Date").max("Time")
df = df.reset_index()
df = df.sort_values(by="Date")
df = df.reset_index(drop=True)

# process dataframe from nasdaq
df1 = df1.sort_values(by="Date")
df1 = df1.reset_index(drop=True)
df1 = df1[["Date", "Close/Last"]]
df1.columns = ["Date", "Close"]
df1["Date"] = df1["Date"].dt.date
df1["Real"] = True

# concat two dataset 
df_max_date = max(df["Date"])
df = pd.concat([df, df1.loc[df1["Date"]>df_max_date]])

# reset index again
df = df.reset_index(drop=True)
df.columns = ["date", "close", "real"]

df.tail()

,date,close,real
3421,2021-05-16,43493.4,True
3422,2021-05-17,44847.2,True
3423,2021-05-18,40569.0,True
3424,2021-05-19,38220.1,True
3425,2021-05-20,40054.0,True


In [3]:
# moving avarage calculation function
def ma(df_source, date, n_days):
    index = df_source.index[df_source["date"] == date][0]
    df = None
    if index == 0:
        df = df_source.iloc[0]
    elif index - n_days >= 0:
        df = df_source.iloc[index - n_days + 1 : index+1]
    else:
        df = df_source.iloc[0 : index + 1]
    return df["close"].mean()

# compare price
def closediff(df_source, col, date):
    index = df_source.index[df_source["date"] == date][0]
    diff = 0
    if index == 0:
        diff = 0
    else:
        diff = df_source.iloc[index][col] - df_source.iloc[index - 1][col]
    return diff

def diffpct(df_source, col, date):
    index = df_source.index[df_source["date"] == date][0]
    diff = 0
    if index == 0:
        diff = 0
    else:
        diff = (df_source.iloc[index][col] - df_source.iloc[index - 1][col])/df_source.iloc[index - 1][col]
    return diff

In [4]:
# get moving averages
ma_days = [5, 10, 20, 30, 60, 90, 180, 240, 360]
df_ma = df.copy()
for ma_day in ma_days:
    column_name = f"ma_{ma_day}"
    df_ma[column_name] = df_ma.apply(lambda x: ma(df_ma, x["date"], ma_day), axis=1)

In [5]:
df["close"].count()
df_ma.to_csv("../data/bitcoin.csv", index=False)

In [6]:
# other price
# gold
df_gold = pd.read_csv("../data/HistoricalData_Gold.csv", parse_dates=["Date"])
df_gold["Date"] = df_gold["Date"].dt.date
df_gold = df_gold[["Date", "Close/Last"]]
df_gold.columns = ["Date", "Gold"]
df_gold["Gold"] = df_gold["Gold"].str[1:]
df_gold["Gold"] = df_gold["Gold"].astype("float")
df_gold.columns = ["date", "gold"]
df_combine = df.merge(df_gold, on="date", how="left")
df_combine["close"].count()

3426

In [7]:
# comp
df_comp = pd.read_csv("../data/HistoricalData_Comp.csv", parse_dates=["Date"])
df_comp["Date"] = df_comp["Date"].dt.date
df_comp = df_comp[["Date", "Close/Last"]]
df_comp.columns = ["date", "comp"]
df_combine = df_combine.merge(df_comp, on="date", how="left")
df_combine["close"].count()

3426

In [8]:
# SP500
df_spx = pd.read_csv("../data/HistoricalData_SPX.csv", parse_dates=["Date"])
df_spx["Date"] = df_spx["Date"].dt.date
df_spx = df_spx[["Date", "Close/Last"]]
df_spx.columns = ["date", "spx"]
df_combine = df_combine.merge(df_spx, on="date", how="left")
df_combine["close"].count()

3426

In [9]:
# Dow Industrials
df_indu = pd.read_csv("../data/HistoricalData_INDU.csv", parse_dates=["Date"])
df_indu["Date"] = df_indu["Date"].dt.date
df_indu = df_indu[["Date", "Close/Last"]]
df_indu.columns=["date", "indu"]
df_combine = df_combine.merge(df_indu, on="date", how="left")
df_combine["close"].count()

3426

In [10]:
# Oil
df_oil = pd.read_csv("../data/HistoricalData_Oil.csv", parse_dates=["Date"])
df_oil["Date"] = df_oil["Date"].dt.date
df_oil = df_oil[["Date", "Close/Last"]]
df_oil.columns = ["date", "oil"]
df_combine = df_combine.merge(df_oil, on="date", how="left")
df_combine["close"].count()

3426

In [11]:
df_combine = df_combine.dropna()

In [12]:
df_combine = df_combine.reset_index(drop=True)

In [13]:
df_combine["close"].count()

2357

In [14]:
df_combine_diff = df_combine.copy()
df_combine_diff["btc_diff"] = df_combine_diff.apply(lambda x: closediff(df_combine_diff, "close", x["date"]), axis=1)
df_combine_diff["gold_diff"] = df_combine_diff.apply(lambda x: closediff(df_combine_diff, "gold", x["date"]), axis=1)
df_combine_diff["comp_diff"] = df_combine_diff.apply(lambda x: closediff(df_combine_diff, "comp", x["date"]), axis=1)
df_combine_diff["spx_diff"] = df_combine_diff.apply(lambda x: closediff(df_combine_diff, "spx", x["date"]), axis=1)
df_combine_diff["indu_diff"] = df_combine_diff.apply(lambda x: closediff(df_combine_diff, "indu", x["date"]), axis=1)
df_combine_diff["oil_diff"] = df_combine_diff.apply(lambda x: closediff(df_combine_diff, "oil", x["date"]), axis=1)

df_combine_diff["btc_diffpct"] = df_combine_diff.apply(lambda x: diffpct(df_combine_diff, "close", x["date"]), axis=1)
df_combine_diff["gold_diffpct"] = df_combine_diff.apply(lambda x: diffpct(df_combine_diff, "gold", x["date"]), axis=1)
df_combine_diff["comp_diffpct"] = df_combine_diff.apply(lambda x: diffpct(df_combine_diff, "comp", x["date"]), axis=1)
df_combine_diff["spx_diffpct"] = df_combine_diff.apply(lambda x: diffpct(df_combine_diff, "spx", x["date"]), axis=1)
df_combine_diff["indu_diffpct"] = df_combine_diff.apply(lambda x: diffpct(df_combine_diff, "indu", x["date"]), axis=1)
df_combine_diff["oil_diffpct"] = df_combine_diff.apply(lambda x: diffpct(df_combine_diff, "oil", x["date"]), axis=1)

In [15]:
df_combine_diff["close"].count()

2357

In [16]:
df_combine_diff.to_csv("../data/combine.csv", index=False)

In [17]:
df_ma.columns

Index(['date', 'close', 'real', 'ma_5', 'ma_10', 'ma_20', 'ma_30', 'ma_60',
       'ma_90', 'ma_180', 'ma_240', 'ma_360'],
      dtype='object')

In [18]:
df_combine_diff.columns

Index(['date', 'close', 'real', 'gold', 'comp', 'spx', 'indu', 'oil',
       'btc_diff', 'gold_diff', 'comp_diff', 'spx_diff', 'indu_diff',
       'oil_diff', 'btc_diffpct', 'gold_diffpct', 'comp_diffpct',
       'spx_diffpct', 'indu_diffpct', 'oil_diffpct'],
      dtype='object')

In [19]:
df1 = pd.read_csv("../data/bitcoin.csv")
print(f"Records: {df1['close'].count()}, Columns: {len(df1.columns)}")
df1.columns

Records: 3426, Columns: 12


Index(['date', 'close', 'real', 'ma_5', 'ma_10', 'ma_20', 'ma_30', 'ma_60',
       'ma_90', 'ma_180', 'ma_240', 'ma_360'],
      dtype='object')

In [20]:
df2 = pd.read_csv("../data/combine.csv")
print(f"Records: {df2['close'].count()}, Columns: {len(df2.columns)}")
df2.columns

Records: 2357, Columns: 20


Index(['date', 'close', 'real', 'gold', 'comp', 'spx', 'indu', 'oil',
       'btc_diff', 'gold_diff', 'comp_diff', 'spx_diff', 'indu_diff',
       'oil_diff', 'btc_diffpct', 'gold_diffpct', 'comp_diffpct',
       'spx_diffpct', 'indu_diffpct', 'oil_diffpct'],
      dtype='object')